# LiqPay API Notebook

## Setup

Load environment variables and code autoreload plugin.

In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv -o .env

Imports

In [ ]:
from os import getenv
from uuid import uuid4
from pprint import pprint
from datetime import datetime, timedelta, UTC
from decimal import Decimal
from webbrowser import open_new_tab

from liqpy.client import Client, AsyncClient
from liqpy.api.exceptions import LiqPayException
from liqpy.dev.testing import TestCard
from liqpy.dev.server import LiqpayServer

Initialize client with public and private keys

In [ ]:
client = Client()
client

## Examples

### Make payments

Pay request with test card

In [ ]:
client.hold(
    order_id=(order_id:= uuid4()),
    amount=Decimal("1.00"),
    currency="USD",
    description="Test Payment",
    **(card_info := TestCard.successful().to_params())
)

In [ ]:
client.complete(order_id=order_id)

In [ ]:
client.pay(
    order_id=(order_id := uuid4()),
    amount=Decimal("2.00"),
    currency="USD",
    description="Test Payment",
    **card_info,
)

Refund the payment

In [ ]:
client.refund(order_id=order_id, amount=Decimal("1.00"))

Get a payment status

In [ ]:
client.status(order_id=order_id)

add info to payment

In [ ]:
client.data(order_id=order_id, info="Lorem Ipsum")

In [ ]:
try:
    client.pay(
        order_id=uuid4(),
        amount=Decimal("1.00"),
        currency="USD",
        description="Test Payment",
        **(failed_card_info := TestCard.failure("9859").to_params()),
    )
except LiqPayException as e:
    print(e.code, e)
    print(e.response)
    pprint(e.details)
    raise e

### Webhooks

Test checkout callback

In [ ]:
checkout_url = client.checkout(
    action=(action := "subscribe"),
    amount=Decimal("1.00"),
    order_id=(order_id := uuid4()),
    description=f"test {action} checkout",
    currency="USD",
    expired_date=(expire := timedelta(minutes=2)),
    # subscribe_date_start=timedelta(days=7),
    subscribe_periodicity="month",
    # result_url="https://example.com/result",
    server_url=(server_url := getenv("SERVER_URL") or str(input("Server URL: "))),
)

print("checkout link\n", checkout_url)
open_new_tab(checkout_url)

timeout = (expire + timedelta(seconds=10)).total_seconds()
with LiqpayServer(client=client, timeout=timeout) as server:
    pprint(server.handle_callback())

In [ ]:
client.unsubscribe(order_id)

### Recurring payments

Create subscription

In [ ]:
client.subscribe(
    amount=Decimal("1.00"),
    order_id=(order_id := uuid4()),
    description="Test Subscribe",
    currency="USD",
    subscribe_periodicity="month",
    subscribe_date_start=timedelta(),
    **(card_info := TestCard.successful().to_params()),
)

Edit a recurring payment

In [ ]:
client.subscription(
    order_id=order_id,
    amount=Decimal("2.00"),
    description="Subsciption with Updated Amount",
    currency="USD",
)

Cancel a recurring payment

In [ ]:
client.unsubscribe(order_id)

### Reports

In [ ]:
date_to = datetime.now()
date_from = date_to - timedelta(hours=1)
print("from:", date_from)
print("to:", date_to)

Get payments report

In [ ]:
result = client.payments(date_from=date_from, date_to=date_to)
print("count:", len(result))
# pprint(result)

Reports in XML (Excel) format

In [ ]:
xml_data = client.reports(date_from=date_from, date_to=date_to, resp_format="xml")

with open("liqpay.xls", "w", encoding="utf-16") as f:
    f.write(xml_data)

Reports in CSV format

In [ ]:
csv_data = client.reports(date_from=date_from, date_to=date_to, resp_format="csv")

with open("liqpay.csv", "w", encoding="utf-16") as f:
    f.write(csv_data)

parse csv reports using standard module

In [ ]:
from csv import DictReader
from liqpy.models.report import Report

reports = [Report.from_dict(item) for item in DictReader(csv_data.splitlines())]
pprint(reports[0])

parse csv reports using pandas

In [ ]:
# !pip install pandas
from io import StringIO
from liqpy.pandas import read_liqpay_csv, ID_TYPE

df = read_liqpay_csv(StringIO(csv_data))
df.dtypes

In [ ]:
df.describe(include=["number"], exclude=[ID_TYPE])

In [ ]:
df.describe(include=["category"])